## Preparação do ambiente

- Definição da sessão spark
- Conexão com o Drive

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
                    .master('local[*]')\
                    .appName('Challenge DataScience - 2 Ed')\
                    .getOrCreate()
spark

In [3]:
from google.colab import drive 
drive.mount('my-drive')

Drive already mounted at my-drive; to attempt to forcibly remount, call drive.mount("my-drive", force_remount=True).


## Manipulações iniciais da base já tratada
  - A partir do dataset já inicialmente tratado a partir do conjunto de dados bruto fornecido no início do projeto, o objetivo será efetuar algumas manipulações básicas (essenciais para os métodos de Machine Learning) e iniciar a construção do modelo de regressão

In [4]:
# começo retomando a base de dados tratados construída anteriormente;
# comparo a base tratada fornecida pelos instrutores da alura com a base construída por mim
# para validação de resultados

df = spark.read.parquet(
    '/content/my-drive/MyDrive/Formações Alura/DataScience/Challenge-2ed/dados-e-arquivos-tratados/dataset-tratado/parquet'
)


In [5]:
df.show(5)

+--------------------+------------+------------+-----------+----------+---------+-------+------+---------+----+-----+--------------------+---------+----------+----------+----+-----+-----+
|                  id|tipo_anuncio|tipo_unidade|   tipo_uso|area_total|area_util|quartos|suites|banheiros|vaga|andar|     caracteristicas|   bairro|      zona|condominio|iptu| tipo|valor|
+--------------------+------------+------------+-----------+----------+---------+-------+------+---------+----+-----+--------------------+---------+----------+----------+----+-----+-----+
|d2e3a3aa-09b5-45a...|       Usado| Apartamento|Residencial|        43|       43|      2|  null|        1|   1|    3|[Academia, Churra...|Paciência|Zona Oeste|       245|null|Venda|15000|
|085bab2c-87ad-452...|       Usado| Apartamento|Residencial|        42|       42|      2|  null|        1|   1|    2|[Churrasqueira, P...|Paciência|Zona Oeste|         0|   0|Venda|15000|
|18d22cbe-1b86-476...|       Usado| Apartamento|Residencial|

In [6]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- area_total: long (nullable = true)
 |-- area_util: long (nullable = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- vaga: long (nullable = true)
 |-- andar: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: long (nullable = true)
 |-- iptu: long (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: long (nullable = true)



- De acordo com o Schema observado, as primeiras manipulações que estaremos interessados em fazer ocorrem com respeito aos datatypes de cada coluna.
  - As colunas "andar", "banheiros", "suites" e "quartos" estão registradas como LongInt. A princípio, para esses casos, apenas o IntegerType já é suficiente.

  - As colunas "area_util", "condominio", "iptu" e "valor", por outro lado, devem ficar registradas com o tipo DoubleType.

In [7]:
from pyspark.sql import functions as f 
from pyspark.sql.types import DoubleType, IntegerType

columns_to_set_DoubleType = ['area_util', 'condominio', 'iptu', 'valor']
columns_to_set_IntegerType = ['andar', 'banheiros', 'suites', 'quartos']

for c in columns_to_set_DoubleType:
  df = df.withColumn(c, f.col(c).cast(DoubleType()))

for c in columns_to_set_IntegerType:
  df = df.withColumn(c, f.col(c).cast(IntegerType()))

df.printSchema()

root
 |-- id: string (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- area_total: long (nullable = true)
 |-- area_util: double (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- vaga: long (nullable = true)
 |-- andar: integer (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: double (nullable = true)
 |-- iptu: double (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: double (nullable = true)



- Nos foi passada a informação de que alguns registros da coluna "caractersticas" são listas vazias. Devemos, portanto, eliminar esses registros do nosso conjunto de dados
  - A ideia será usar o método ".getItem(0)" para verificar quais registros têm esse primeiro valor nulo. Se ele for nulo, que dizer que a lista da linha correspondente é vazia


In [8]:
# contando quantos elementos contém listas vazias na coluna de características
df.select('*').where(f.isnull( f.col('caracteristicas').getItem(0) )).count()

12665

- Como são muitos dados (~10 000, mesma ordem de grandeza da quantidade total de registros) não podemos simplesmente descartá-los do nosso dataset; as demais colunas associadas a essas linhas podem conter informações relevantes para a construção posterior dos modelos de regressão.
- Para contornar esse problema, portanto, ajusto todas essas colunas com valor nulo (null)

In [9]:
# outra maneira de detectar esses caras utilizando a função "size" do pyspark.sql
df.select('*').filter( f.size( f.col('caracteristicas') ) == 0 ).count()

12665

In [10]:
# verificação dos arrays vazios (antes do tratamento)
df.select('caracteristicas').distinct().orderBy('caracteristicas', ascending=True).show(5)

+--------------------+
|     caracteristicas|
+--------------------+
|                  []|
|          [Academia]|
|[Academia, Animai...|
|[Academia, Animai...|
|[Academia, Animai...|
+--------------------+
only showing top 5 rows



In [11]:
# Faço a atualização desses dados utilizando o case when/else (f.when().otherwise, no caso do pyspark)

# detalhes da logica abaixo: quando o "len" da coluna caracteristicas for 0 (i.e., quando o array do campo correspondente for nulo),
# adicionamos o valor null (None para o dataframe spark no python); caso contrário, ajustamos o registro da própria coluna
df = df.withColumn('caracteristicas', f.when( f.size( f.col('caracteristicas') ) == 0, None ).otherwise( f.col('caracteristicas') ) )

# Poderíamos também estruturar esse mesmo update utilizando o método "getIem(0)" (verificando quando valor logo do primeiro index fosse nulo, o que 
# indicaria uma lista vazia)
# (utilizo uma sintaxe diferente no no argumento de aplicação das funções de tratamento apenas por prática)
#df = df.withColumn('caracteristicas', f.when( df['caracteristicas'].getItem(0).isNull(), None ).otherwise( df['caracteristicas'] ) )

In [12]:
#teste = df.select('*')
#teste = teste.withColumn('caracteristicas', f.when( f.col('caracteristicas').getItem(0).isNull(), 0 ).otherwise( f.col('caracteristicas') ) )

# o método "case when / else " (when / otherwise, no pyspark) não funcionou pois estamos colocando dois tipos de dados diferentes como saída:
# inteiro se a condição verificar verdadeira e array se a condição verificar falsa;
# a lógica está correta mas o spark não consegue efetuar esse tratamento

In [13]:
# validação da função de tratamento aplicada
df.select('caracteristicas').distinct().orderBy('caracteristicas', ascending=True).show(5)

+--------------------+
|     caracteristicas|
+--------------------+
|                null|
|          [Academia]|
|[Academia, Animai...|
|[Academia, Animai...|
|[Academia, Animai...|
+--------------------+
only showing top 5 rows



- Verifico a presença de registros nulos ao longo das colunas do nosso dataset e efetuo os tratamentos correspondentes

In [14]:
array_columns_to_check = df.columns

array_columns_to_check.remove('caracteristicas') 
# excluo a coluna de caracteristicas desse processo automatizado de contagem, mas sabemos já sabemos que ela contém um total de 
# 12 665 dados nulos (ajustes efetuados na célula anterior)

df.select([ f.count( f.when( f.isnull(col) | f.isnan(col) , 1 ) ).alias(col) for col in array_columns_to_check ] ).show()

+---+------------+------------+--------+----------+---------+-------+------+---------+----+-----+------+----+----------+----+----+-----+
| id|tipo_anuncio|tipo_unidade|tipo_uso|area_total|area_util|quartos|suites|banheiros|vaga|andar|bairro|zona|condominio|iptu|tipo|valor|
+---+------------+------------+--------+----------+---------+-------+------+---------+----+-----+------+----+----------+----+----+-----+
|  0|           0|           0|       0|      9186|        0|      0|  5544|        0|3008|    0|     0|   0|      2347|7155|   0|    0|
+---+------------+------------+--------+----------+---------+-------+------+---------+----+-----+------+----+----------+----+----+-----+



In [15]:
# verifico se a contagem anterior se refere somente aos dados null, ou se dados tipo nan contribuiram para o resultado final
df.select( [f.count( f.when( f.isnull(c), 1 ) ).alias(c) for c in array_columns_to_check] ) .show()

+---+------------+------------+--------+----------+---------+-------+------+---------+----+-----+------+----+----------+----+----+-----+
| id|tipo_anuncio|tipo_unidade|tipo_uso|area_total|area_util|quartos|suites|banheiros|vaga|andar|bairro|zona|condominio|iptu|tipo|valor|
+---+------------+------------+--------+----------+---------+-------+------+---------+----+-----+------+----+----------+----+----+-----+
|  0|           0|           0|       0|      9186|        0|      0|  5544|        0|3008|    0|     0|   0|      2347|7155|   0|    0|
+---+------------+------------+--------+----------+---------+-------+------+---------+----+-----+------+----+----------+----+----+-----+



In [16]:
# preenchimento dos dados nulos com valores 0
df = df.na.fill(0)

In [17]:
# efetuo novamente a contagem por fins de validação
df.select( [f.count( f.when( f.isnull(c), 1 ) ).alias(c) for c in array_columns_to_check] ) .show()

+---+------------+------------+--------+----------+---------+-------+------+---------+----+-----+------+----+----------+----+----+-----+
| id|tipo_anuncio|tipo_unidade|tipo_uso|area_total|area_util|quartos|suites|banheiros|vaga|andar|bairro|zona|condominio|iptu|tipo|valor|
+---+------------+------------+--------+----------+---------+-------+------+---------+----+-----+------+----+----------+----+----+-----+
|  0|           0|           0|       0|         0|        0|      0|     0|        0|   0|    0|     0|   0|         0|   0|   0|    0|
+---+------------+------------+--------+----------+---------+-------+------+---------+----+-----+------+----+----------+----+----+-----+



In [18]:
# verifico se os registros da coluna de caracteristicas foram corrigidos
df.select('caracteristicas').distinct().orderBy('caracteristicas', ascending=True).show(5)

+--------------------+
|     caracteristicas|
+--------------------+
|                null|
|          [Academia]|
|[Academia, Animai...|
|[Academia, Animai...|
|[Academia, Animai...|
+--------------------+
only showing top 5 rows



- Ao que podemos ver os registros nulos da coluna de caracteristicas não estão sendo corrigidos pelas nossas ferramentas usuais. Por enquanto isso não é um problema, pois a nossa ideia será quebrar os registros desses arrays em dummy variables no momento em que estivermos preparando o dataset para construção do modelo de regressão.

In [19]:
df.select('zona').distinct().collect()

[Row(zona='Zona Norte'),
 Row(zona='Zona Oeste'),
 Row(zona='Zona Central'),
 Row(zona='Zona Sul'),
 Row(zona='')]

In [20]:
# começo criando as dummy variables para a coluna de zonas
df_zona = df\
            .groupBy('id')\
            .pivot('zona')\
            .agg(f.lit(1))\
            .na\
            .fill(0)

df_zona.show(5)


+--------------------+---+------------+----------+----------+--------+
|                  id|   |Zona Central|Zona Norte|Zona Oeste|Zona Sul|
+--------------------+---+------------+----------+----------+--------+
|4e47e4d4-3326-4eb...|  0|           0|         0|         0|       1|
|02fba6ef-a691-442...|  0|           0|         0|         1|       0|
|fc03c1a9-8bbb-41a...|  0|           0|         1|         0|       0|
|3dd5d200-0a7f-43d...|  0|           0|         0|         0|       1|
|82707939-71bd-40c...|  0|           0|         0|         0|       1|
+--------------------+---+------------+----------+----------+--------+
only showing top 5 rows



In [21]:
df_zona = df_zona.drop('')
# o método ".drop" está sendo incluído para excluir uma coluna vazia que está sendo criada, correspondente a uma zona registrada como " '' "

df_zona.show(5)

+--------------------+------------+----------+----------+--------+
|                  id|Zona Central|Zona Norte|Zona Oeste|Zona Sul|
+--------------------+------------+----------+----------+--------+
|4e47e4d4-3326-4eb...|           0|         0|         0|       1|
|02fba6ef-a691-442...|           0|         0|         1|       0|
|fc03c1a9-8bbb-41a...|           0|         1|         0|       0|
|3dd5d200-0a7f-43d...|           0|         0|         0|       1|
|82707939-71bd-40c...|           0|         0|         0|       1|
+--------------------+------------+----------+----------+--------+
only showing top 5 rows



- Agora que o dataframe auxiliar de dummy variables correspondente à coluna "zona" está feito, desenvolvo um tratamento da coluna "características" (formato array)

In [22]:
# defino um dataframe auxiliar incluindo somente as colunas que vamos trabalhar, com a finalidade de não sujar o nosso dataframe original
df_aux = df\
            .select('id', 'caracteristicas')

# incluo uma nova coluna nesse dataframe utilizando o método ".explode()", que basicamente funciona extraindo a os arrays da coluna de características
# e colocando-os numa mesma coluna para cada id correspondente.

# em outras palavras: se uma coluna com id "x" possuía inicialmente 5 valores dentro do array da coluna 'caracteristicas', então
# serão criadas 5 novas linhas com o id "x", uma para cada valor dentro do array da coluna 'caracteristicas'.

df_aux = df_aux.withColumn('caracteristicas_values', f.explode( f.col('caracteristicas') ))
df_aux.show(n = 10, truncate=False)

+------------------------------------+-------------------------------------------------------------------------------------------------------------------------------+----------------------+
|id                                  |caracteristicas                                                                                                                |caracteristicas_values|
+------------------------------------+-------------------------------------------------------------------------------------------------------------------------------+----------------------+
|d2e3a3aa-09b5-45a0-9dcd-918847cd3ca3|[Academia, Churrasqueira, Playground, Salão de festas, Condomínio fechado, Portão eletrônico, Portaria 24h, Animais permitidos]|Academia              |
|d2e3a3aa-09b5-45a0-9dcd-918847cd3ca3|[Academia, Churrasqueira, Playground, Salão de festas, Condomínio fechado, Portão eletrônico, Portaria 24h, Animais permitidos]|Churrasqueira         |
|d2e3a3aa-09b5-45a0-9dcd-918847cd3ca3|[Academia, C

In [23]:
# agora efetuo um pivot da nova coluna criada e defino um novo dataframe auxiliar
df_caracteristicas = df_aux\
                          .groupBy('id')\
                          .pivot('caracteristicas_values')\
                          .agg(f.lit(1))\
                          .na\
                          .fill(0)

df_caracteristicas.show(10)

+--------------------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|                  id|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|
+--------------------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|fd96bbd5-d631-416...|       1|                 1|            1|                 1|       1|      1|         1|           1|                1|              1|
|bfffedfe-99e7-4ae...|       0|                 1|            1|                 1|       1|      0|         1|           0|                1|              1|
|fcb67af3-5601-415...|       1|                 1|            1|                 0|       0|      1|         1|           0|                0|              1|
|afecddff-f4cc-4ab...|       1|               

- Uma vez feitos esses tratamentos, junto tudo no nosso dataset inicial

In [24]:
df = df.join(other= df_zona , on= 'id', how= 'inner')
df = df.join(other= df_caracteristicas, on= 'id', how= 'inner')

df.show(10)

+--------------------+------------+------------+-----------+----------+---------+-------+------+---------+----+-----+--------------------+--------------------+----------+----------+------+-----+---------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|                  id|tipo_anuncio|tipo_unidade|   tipo_uso|area_total|area_util|quartos|suites|banheiros|vaga|andar|     caracteristicas|              bairro|      zona|condominio|  iptu| tipo|    valor|Zona Central|Zona Norte|Zona Oeste|Zona Sul|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|
+--------------------+------------+------------+-----------+----------+---------+-------+------+---------+----+-----+--------------------+--------------------+----------+----------+------+-----+---------+------------+----------+--

In [26]:
# todos os tratamentos efetuados anteriormente são essenciais para aplicação em treino/teste nos modelos de ML.
# para dinamizar e facilitar os processos futuros, salvo essa base e deixo guardada para quando mais for necessário

df.write.parquet(
    '/content/my-drive/MyDrive/Formações Alura/DataScience/Challenge-2ed/dados-e-arquivos-tratados/dataset_tratado_para_ML_parquet'
)

## Construindo e estudando os modelos de Machine Learning
- Nessa seção estudaremos os 3 principais tipos de modelos de regressão para bases de dados tratadas (Regressão Linear, Árvore de Decisão e Random Forest) com a finalidade de verificar qual deles melhor se adequa à nossa base de dados